### Enter full names of group members:

##### Name A:
##### Name B:

In [65]:
import math

import numpy
import numpy as np
from sympy import prime
from pathlib import Path  # for paths of files
import csv
import copy
import random
from sklearn.metrics.pairwise import cosine_similarity

# ANSI escape codes for colors
class colors:
    red = '\033[91m'
    green = '\033[92m'
    blue = '\033[94m'
    end = '\033[0m'  

### 1. DGIM

#### 1.1. DGIM algorithm

In [66]:
# Default DGIM parameters

stream_path = 'data/my_stream.txt'

# The window size
N = 500 

In [67]:
def dgim_algorithm(stream_path, N):
    
    # Create the buckets and initialize the timestamp
    pos=1
    bucket_list=[[]]


    # Loop through the entire data stream, one bit at a time
    with open(stream_path) as f:
        while True:
            bit = f.read(1)
            
            # Clause to break while loop at the end of the stream
            if not bit:
                break
            
            if bit=="1":
                bucket_list[0].append(pos)
                for b_index, bucket in enumerate(bucket_list):
                    if len(bucket)==3:
                        if len(bucket_list)>b_index+1:
                            bucket_list[b_index+1].append(bucket[1])
                            bucket_list[b_index]=[bucket[2]]
                        else:
                            bucket_list.append([bucket[1]])
                            bucket_list[b_index]=[bucket[2]]
                    else:
                        break

            # Failsafe
            if len(bucket_list[0])>4:
                break



            #We noticed alot of line-breaks in the my_stream.txt file. 103 of them. 
            #If you remove all of them the end time-stamp will become 1010000
            #If you only remove the three trailing last ones the end time-stamp will become 1010099
            #The answer only becomes truly correct if you remove all of them, so we filtered them out of the buckets like this.
            if bit=="1" or bit =="0":
                pos+=1

    end_time_stamp=pos-1
            
                            
    return bucket_list, end_time_stamp

In [68]:
bucket = dgim_algorithm(stream_path, N)

In [69]:
print(f"The updated list of timestamps buckets from DGIM algorithm: \n {bucket[0]}")
print(f"The end timestamp: {bucket[1]}")   

The updated list of timestamps buckets from DGIM algorithm: 
 [[1010000], [1009992, 1009997], [1009984, 1009990], [1009964, 1009976], [1009945], [1009907], [1009722, 1009847], [1009335, 1009589], [1008598, 1009104], [1007062, 1008118], [1006021], [999737, 1003947], [987359, 995660], [979037], [962470], [862888, 929247], [663800, 796538], [265569, 530961]]
The end timestamp: 1010000


#### 1.2. Query the Bucket 

In [70]:
def actual_count(stream_path, k):
    stream_list = []
    with open(stream_path, 'r') as file:
        for line in file:
            stream_list.extend(list(map(int, line.strip())))

    # Convert the list into a numpy array
    stream_array = np.array(stream_list)
    
    return int(np.sum(stream_array[-k:]))

In [71]:
def dgim_query(bucket, N, k): 
    # Extract the buckets and the end timestamp
    bucket_list, end_time_stamp = bucket
   
    one_count=0
    last_added=0
    stamp=end_time_stamp
    for bucket_index, bibuck in enumerate(bucket_list):
        for stamp in reversed(bibuck):
            if stamp<=end_time_stamp-k:
                one_count-=last_added/2
                break
            else:
                last_added=2**bucket_index
                one_count+=last_added
        if stamp<=end_time_stamp-k+1:
            break
    
    return math.ceil(one_count)

In [72]:
# List of queries
K = [10, 50, 100, 300, 500] 

In [73]:
print("---------------------------------------------------------------")
for k in K:
    dgim_count = dgim_query(bucket, 500, k)
    true_count = actual_count(stream_path, k)
    
    print(f"The total 1s in the last {k} bits by DGIM: {dgim_count}")
    print(f"The true count of 1s in the last {k} bits: {true_count}")
    print(f"The DGIM error for predicted 1s in the last {k} bits: \
    {round(abs(100*(dgim_count-true_count))/true_count,2)} %")
    print("---------------------------------------------------------------")

---------------------------------------------------------------
The total 1s in the last 10 bits by DGIM: 4
The true count of 1s in the last 10 bits: 5
The DGIM error for predicted 1s in the last 10 bits:     20.0 %
---------------------------------------------------------------
The total 1s in the last 50 bits by DGIM: 25
The true count of 1s in the last 50 bits: 26
The DGIM error for predicted 1s in the last 50 bits:     3.85 %
---------------------------------------------------------------
The total 1s in the last 100 bits by DGIM: 61
The true count of 1s in the last 100 bits: 51
The DGIM error for predicted 1s in the last 100 bits:     19.61 %
---------------------------------------------------------------
The total 1s in the last 300 bits by DGIM: 173
The true count of 1s in the last 300 bits: 150
The DGIM error for predicted 1s in the last 300 bits:     15.33 %
---------------------------------------------------------------
The total 1s in the last 500 bits by DGIM: 269
The true 

### 2. Bloom filters

In [74]:
# Username data for the creation of bloom filters - B
data_file = (Path("data/bloom_username").with_suffix('.csv'))

# Test data to check the functionality and false positive rate
test1_file = (Path("data/test1_username").with_suffix('.csv'))
test2_file = (Path("data/test2_username").with_suffix('.csv'))

# Default bloom filter parameters
bloom_size = 1500000 # parameter N
h = 3 # number of hash functions

In [75]:
# create an array of bloom filter with zeros
B = np.zeros(bloom_size)

In [76]:
B

array([0., 0., 0., ..., 0., 0., 0.])

#### 2.1. Create Bloom filter

In [77]:
def generatePrimes(n):
    i = 3
    primes=[2]
    flag = False
    while(len(primes) < n):
        flag = True
        for j in primes:
            if math.floor(math.sqrt(i)) + 1 <= j:
                break
            elif (i%j == 0):
                flag = False
                break
        if(flag):
            primes.append(i)
        i+=1
    
    return primes

def hash_function(p,N):
    return lambda s: (sum([ord(s[i])*(p**(i+1)) for i in range(len(s))])%N)

def generate_hash(h, N):
    hash_list = []

    prime_list_length=math.floor(math.sqrt(N))
    #Generate h different random primes to make hash functions more random
    seeds=random.sample(range(0, prime_list_length), h)
    primes=generatePrimes(prime_list_length)
    primes=[primes[seeds[i]] for i in range(h)]
    

    for p in range(h):
        func = hash_function(primes[p],N)
        hash_list.append(func)
    return hash_list

def hash_function(p,N):
    return lambda s: (sum([ord(s[i])*(p**(i+1)) for i in range(len(s))])%N)

In [78]:
hashes = generate_hash(h, bloom_size)

In [79]:
def create_bloom_filter(B, hashes, data):
    with data.open(encoding="utf-8") as f:
        for name in f:
            for hash in hashes:
                index=hash(name.strip())
                B[index]=1
    return B

In [80]:
bloom_array = create_bloom_filter(B, hashes, data_file)

In [81]:
bloom_array


array([0., 1., 1., ..., 1., 0., 0.])

#### 2.2. Verify usernames

In [82]:
def single_verify_username(bloom_array, hashes, new_user):
    code=0
    existing_entries=1
    for hash in hashes:
        if bloom_array[hash(new_user.strip())]==0:
            existing_entries=0
            break
    return existing_entries

In [83]:
# Feel free to test different usernames here

new_username = "hubble2010"

#new_username = "ShambaTDT4305"

In [84]:
user_code = single_verify_username(bloom_array, hashes, new_username)

In [85]:
if user_code == 1:
    print(colors.red + f"Username {new_username} has been taken. Try again!" + colors.end)
elif user_code == 0:
    print(colors.green + f"Username {new_username} is available. Congrats!" + colors.end)
else:
    print(colors.blue + f"Wrong pass code. Please reverify!" + colors.end)  

Username hubble2010 has been taken. Try again!


In [86]:
def group_verify_username(bloom_array, hashes, data):
    # Initialize counts
    total_name = 0
    taken_name = 0
    
    with data.open(encoding="utf-8") as f:
        for name in f:
            total_name+=1
            taken_name+=single_verify_username(bloom_array, hashes, name)
            
    return round(taken_name/total_name*100,2)   

In [87]:
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test1_file)
print(f"Percentage of username seen before from test 1: {user_total}%")
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test2_file)
print(f"Percentage of username seen before from test 2: {user_total}%")
print("----------------------------------------------------------")


----------------------------------------------------------
Percentage of username seen before from test 1: 100.0%
----------------------------------------------------------
Percentage of username seen before from test 2: 23.78%
----------------------------------------------------------


### 3. Flajolet-Martin

In [88]:
def r(a):
    for i in range(a.bit_length()):
        if a & (1 << i):
            return i   
    return 0

def flajolet_martin(input_stream):
    R = 0  # Initialize maximum rightmost zero bit position to 0
    h=lambda x: (6*x+1)%5
    # To-do! Define hash function h(x) = 6x + 1 mod 5
    
    for element in input_stream:
        temp_r=r(h(element))
        R=max(temp_r,R)
    # To-do! Iterate over the input stream and update maximum rightmost zero bit position
    

    # Estimate the number of distinct elements
    distinct_estimate = 2 ** R

    return distinct_estimate

In [89]:
# Input stream
input_stream1 = [1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1]
input_stream2 = [1, 3, 2, 1, 2, 3, 4, 3, 1, 2, 3, 1]

# Run the Flajolet-Martin algorithm
distinct_estimate1 = flajolet_martin(input_stream1)
distinct_estimate2 = flajolet_martin(input_stream2)

# Print the estimated number of distinct elements
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 1:", distinct_estimate1)
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 2:", distinct_estimate2)
print("-----------------------------------------------------")

-----------------------------------------------------
Distinct elements (estimated) in input stream 1: 2
-----------------------------------------------------
Distinct elements (estimated) in input stream 2: 4
-----------------------------------------------------


### 4. Adword 

#### 4.1. Greedy Algorithm

In [90]:
# User queries
queries = ["big data", "big data", "big data","bloom filters", "bloom filters", "bloom filters",
           "flajolet martin", "flajolet martin", "flajolet martin", "dgim algorithm", "dgim algorithm", "dgim algorithm"]

In [91]:
# Company A B C and D keywords and budget $$$
global_companies = {
        'A': ["big data", "bloom filters", 3],
        'B': ["flajolet martin", 3],
        'C': ["flajolet martin", "dgim algorithm", 3],
        'D': ["big data", 3],
    }

In [92]:
def greedy_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    for q in queries:
        order = random.sample(list(local_companies.keys()), len(local_companies))
        match=False
        for company in order:
            bids = local_companies[company]
            for i in range(0, len(bids)-1):
                if bids[i] == q:
                    if bids[-1] > 0:
                        match=True
                        bids[-1] -= 1
                        revenue += 1
                        break
            if match:
                break

    
    return revenue




In [93]:
total_revenue = 0
total_trials = 10
print("Starting trials using Greedy Algorithm...")
print("------------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = greedy_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("------------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Greedy Algorithm...
------------------------------------------------
Trial 1 - Revenue generated: 8
Trial 2 - Revenue generated: 9
Trial 3 - Revenue generated: 9
Trial 4 - Revenue generated: 8
Trial 5 - Revenue generated: 9
Trial 6 - Revenue generated: 8
Trial 7 - Revenue generated: 7
Trial 8 - Revenue generated: 8
Trial 9 - Revenue generated: 7
Trial 10 - Revenue generated: 10
------------------------------------------------
Average revenue generated for all trials:  8.3


#### 4.2. Balance Algorithm

In [94]:
def balance_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    for q in queries:
        candidate_companies=[]
        buy_in=0
        for company, bids in local_companies.items():
            if q in bids and bids[-1]>0:
                if bids[-1]>buy_in:
                    buy_in=bids[-1]
                    candidate_companies=[company]
                elif bids[-1]==buy_in:
                    candidate_companies.append(company)
        if candidate_companies:
            local_companies[candidate_companies[random.randint(0,len(candidate_companies)-1)]][-1]-=1
            revenue+=1


    
    return revenue

In [95]:
total_revenue = 0
total_trials = 10
print("Starting trials using Balance Algorithm...")
print("-------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = balance_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("-------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Balance Algorithm...
-------------------------------------------
Trial 1 - Revenue generated: 10
Trial 2 - Revenue generated: 9
Trial 3 - Revenue generated: 9
Trial 4 - Revenue generated: 9
Trial 5 - Revenue generated: 9
Trial 6 - Revenue generated: 8
Trial 7 - Revenue generated: 9
Trial 8 - Revenue generated: 9
Trial 9 - Revenue generated: 9
Trial 10 - Revenue generated: 9
-------------------------------------------
Average revenue generated for all trials:  9.0


### 5. Recommender System

In [96]:
# Ratings matrix (each row corresponds to a movie, and each column corresponds to a user)
ratings_matrix = np.array([
    [1, 0, 3, 0, 0, 5, 0, 0, 5, 0, 4, 0],
    [0, 0, 5, 4, 0, 0, 4, 0, 0, 2, 1, 3],
    [2, 4, 0, 1, 2, 0, 3, 0, 4, 3, 5, 0],
    [0, 2, 4, 0, 5, 0, 0, 4, 0, 0, 2, 0],
    [0, 0, 4, 3, 4, 2, 0, 0, 0, 0, 2, 5],
    [1, 0, 3, 0, 3, 0, 0, 2, 0, 0, 4, 0]
])

#### 5.1. User-User Collaborative Filtering

In [97]:
def find_similar_vectors(rating_matrix,target_index,neigh):
    similarity_scores=[]
    user_vector = rating_matrix[target_index]
    for i in range(len(rating_matrix)):
        if i==target_index:
            continue
        similarity=cosine_similarity([user_vector],[rating_matrix[i]])[0][0]
        similarity_scores.append([i,similarity])
    x=lambda sim: sim[1]
    similarity_scores.sort(key=x)
    return similarity_scores[-neigh:]

def user_cf(rate_m, tup_mu, neigh):
    tup_mu=[tup_mu[0]-1,tup_mu[1]-1]
    # To-do! implement a user-user CF using cosine similarity as distance measure
    similar_users=find_similar_vectors(np.transpose(rate_m),tup_mu[1],neigh)

    prediction=sum([rate_m[tup_mu[0]][user[0]]*user[1] for user in similar_users]) / sum([user[1] for user in similar_users])

    return prediction

In [98]:
# List of tuple of movie rating by users to be predicted e.g (1, 5) refers to the rating of movie 1 by user 5
list_mu_query = [(1, 5), (3, 3)]

# Neighbor selection (|N|)
neigh = 2

In [107]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = user_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (User-User CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 1.4158544560666992 (User-User CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 1.4938710151784431 (User-User CF)
-----------------------------------------------------------------


#### 5.2. Item-Item Collaborative Filtering

In [123]:
def item_cf(rate_m, tup_mu, neigh):
    rate_m_2 = rate_m.copy().astype(np.float32)
    print(rate_m_2)
    # To-do! implement a item-item CF using cosine similarity as distance measure
    tup_mu=[tup_mu[0]-1,tup_mu[1]-1]
    for row, movie in enumerate(rate_m_2):
        average = np.average(movie)
        non_zero_avg = average * (len(movie)/np.count_nonzero(movie))
        for col, user in enumerate(movie):
            if rate_m[row][col]:
                rate_m_2[row][col] = user-non_zero_avg
    
    # To-do! implement a item-item CF using cosine similarity as distance measure
    similar_movies = find_similar_vectors(rate_m_2, tup_mu[0], neigh)
    prediction=sum([rate_m[movie[0]][tup_mu[1]]*movie[1] for movie in similar_movies]) / sum([movie[1] for movie in similar_movies])
    return prediction

In [124]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = item_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (Item-Item CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
[[1. 0. 3. 0. 0. 5. 0. 0. 5. 0. 4. 0.]
 [0. 0. 5. 4. 0. 0. 4. 0. 0. 2. 1. 3.]
 [2. 4. 0. 1. 2. 0. 3. 0. 4. 3. 5. 0.]
 [0. 2. 4. 0. 5. 0. 0. 4. 0. 0. 2. 0.]
 [0. 0. 4. 3. 4. 2. 0. 0. 0. 0. 2. 5.]
 [1. 0. 3. 0. 3. 0. 0. 2. 0. 0. 4. 0.]]
The predicted rating of movie 1 by user 5: 2.586406888682535 (Item-Item CF)
-----------------------------------------------------------------
[[1. 0. 3. 0. 0. 5. 0. 0. 5. 0. 4. 0.]
 [0. 0. 5. 4. 0. 0. 4. 0. 0. 2. 1. 3.]
 [2. 4. 0. 1. 2. 0. 3. 0. 4. 3. 5. 0.]
 [0. 2. 4. 0. 5. 0. 0. 4. 0. 0. 2. 0.]
 [0. 0. 4. 3. 4. 2. 0. 0. 0. 0. 2. 5.]
 [1. 0. 3. 0. 3. 0. 0. 2. 0. 0. 4. 0.]]
The predicted rating of movie 3 by user 3: 3.0 (Item-Item CF)
-----------------------------------------------------------------


### Provide concise answers to all 5 cases in the Project 3 description below

#### Case 1

In [110]:
# While the number of buckets is log(N) over N bits, the timestamp we have to save for each bucket will also need a space complexity of Log(N). Each timestamp increases in size logarithmically as well. 

#### Case 2

In [126]:
# It is possible that the name is flagged as taken even though it is available. The Bloom Filter algorithm uses hash functions that checks if a name is already taken, and there will always be a chance that two different usernames hashes into the same output. With multiple hash functions, the chance decreases, but it will still be possible. In this case the Bloom Filter will flag a free username as taken. It would be possible for a site-admin to manually use any search function on the given username in the database to see if its actually taken. 

# In the opposite case, where a friend has claimed that a username flagged as available is already taken, there should be no possible way for this to happen in the Bloom Filter algorithm. This is because the hash function for a username will always give the same output no matter what, and when that username is then flagged as taken, there is no way for the username to have the flag of available. 

#### Case 3

In [112]:
# Enter answer here

#### Case 4

In [113]:
# The greedy algorithm with the given input will have:
# Worst possible outcome: 6
# Best possible outcome: 12
# Competitive ratio: 1/2

# The balance algorithm with the given input will have:
# Worst possible outcome: 8
# Best possible outcome: 12
# Competitive ratio: The general competitive ratio for the balance algorithm is 1-1/e=0.63. In our case the competitive ratio is 8/12=0.66, because we have so few companies in our example. 

#### Case 5

In [114]:
# Enter answer here